## Ejercicio 1: Filtrado Colaborativo Basado en Ítems

Implementa un sistema de recomendación basado en ítems utilizando la matriz de calificaciones proporcionada en el ejemplo práctico 1. Recomendale al Usuario 'B' dos productos que aún no ha calificado.

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Cargar datos de ejemplo (matriz de calificaciones)
data = {
    'Usuario': ['A', 'B', 'C', 'D', 'E'],
    'Producto1': [5, 3, 0, 1, 4],
    'Producto2': [4, 0, 0, 1, 2],
    'Producto3': [1, 1, 0, 5, 0],
    'Producto4': [0, 0, 5, 4, 0],
    'Producto5': [0, 3, 4, 0, 0],
}

df = pd.DataFrame(data)
df.set_index('Usuario', inplace=True)

In [2]:
df

,Producto1,Producto2,Producto3,Producto4,Producto5
Usuario,,,,,
A,5,4,1,0,0
B,3,0,1,0,3
C,0,0,0,5,4
D,1,1,5,4,0
E,4,2,0,0,0


In [3]:
# Calcular la similitud entre productos usando la similitud del coseno
# Transponemos el dataframe para obtener cada producto como vector
item_similarity = cosine_similarity(df.T)
item_similarity_df = pd.DataFrame(item_similarity, index=df.columns, columns=df.columns)

print("Matriz de similitud entre productos:")
print(item_similarity_df)

# Función para predecir la calificación de un usuario para un producto no evaluado
def predict_rating(user, item):
    # Obtener las calificaciones del usuario
    user_ratings = df.loc[user]
    # Consideramos solo aquellos productos que ya han sido evaluados (asumimos 0 como "no evaluado")
    rated_items = user_ratings[user_ratings != 0]
    
    numerator = 0
    denominator = 0
    # Calcular la suma ponderada de las calificaciones según la similitud entre el producto objetivo y los productos evaluados
    for other_item, rating in rated_items.items():
        similarity = item_similarity_df.loc[item, other_item]
        numerator += similarity * rating
        denominator += abs(similarity)
        
    if denominator == 0:
        return 0
    return numerator / denominator

# Recomendaciones para el Usuario 'B'
user = 'B'
# Identificar los productos que el usuario B no ha calificado (valor 0)
products_not_rated = df.columns[df.loc[user] == 0]

# Predecir la calificación para cada producto no evaluado
predictions = {}
for item in products_not_rated:
    pred = predict_rating(user, item)
    predictions[item] = pred

# Ordenar las predicciones de mayor a menor
recommendations = sorted(predictions.items(), key=lambda x: x[1], reverse=True)

print(f"\nRecomendaciones para el usuario {user}:")
for item, rating in recommendations:
    print(f"{item}: calificación predicha {rating:.2f}")


Matriz de similitud entre productos:
           Producto1  Producto2  Producto3  Producto4  Producto5
Producto1   1.000000   0.886142   0.350329   0.087475   0.252050
Producto2   0.886142   1.000000   0.377964   0.136320   0.000000
Producto3   0.350329   0.377964   1.000000   0.601113   0.115470
Producto4   0.087475   0.136320   0.601113   1.000000   0.624695
Producto5   0.252050   0.000000   0.115470   0.624695   1.000000

Recomendaciones para el usuario B:
Producto2: calificación predicha 2.40
Producto4: calificación predicha 2.08


## Ejercicio 2: Evaluación de Métricas
Dada la siguiente matriz de calificaciones reales y predichas para un conjunto de usuarios y productos, calcula el RMSE y el MAE.

|Usuario|	Producto1 (Real)|	Producto1 (Pred)|	Producto2 (Real)|	Producto2 (Pred)|
|---|---|---|---|---|
|A|	5|	4.5|	3|	3.5|
|B|	2|	2.0|	4|	3.8|
|C|	0|	1.2|	5|	4.9|
|D|	3|	3.5|	0|	0.2|

In [4]:
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Datos de calificaciones reales y predichas:
# Para Producto1:
reales_p1 = [5, 2, 0, 3]
predichos_p1 = [4.5, 2.0, 1.2, 3.5]

# Para Producto2:
reales_p2 = [3, 4, 5, 0]
predichos_p2 = [3.5, 3.8, 4.9, 0.2]

# Concatenamos ambas listas para obtener todos los valores
y_real = reales_p1 + reales_p2
y_pred = predichos_p1 + predichos_p2

# Cálculo del RMSE
mse = mean_squared_error(y_real, y_pred)
rmse = math.sqrt(mse)

# Cálculo del MAE
mae = mean_absolute_error(y_real, y_pred)

print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")


RMSE: 0.534
MAE: 0.400


### Ejercicio 3: Filtrado Basado en Contenido Avanzado
Ampliando el ejemplo práctico 2, incluye una nueva columna que clasifique los productos en categorías (por ejemplo, 'Electrónica', 'Cocina', 'Literatura', 'Moda', 'Juguetes'). Utiliza esta información para mejorar el sistema de recomendación basado en contenido, de manera que solo recomiende productos de la misma categoría que el producto objetivo.

In [5]:
# Diccionario que asigna una categoría a cada producto
categorias = {
    'Producto1': 'Electrónica',
    'Producto2': 'Cocina',
    'Producto3': 'Literatura',
    'Producto4': 'Moda',
    'Producto5': 'Juguetes'
}

# (Opcional) Crear un DataFrame con la información de los productos y sus categorías
productos_info = pd.DataFrame(list(categorias.items()), columns=['Producto', 'Categoría'])
productos_info.set_index('Producto', inplace=True)
print("Información de productos con categorías:")
print(productos_info)

# Calcular la similitud entre productos utilizando la similitud del coseno.
# Se transponen los datos para que cada producto sea representado como un vector.
item_similarity = cosine_similarity(df.T)
item_similarity_df = pd.DataFrame(item_similarity, index=df.columns, columns=df.columns)

print("\nMatriz de similitud entre productos:")
print(item_similarity_df)

# Función para predecir la calificación de un usuario para un producto objetivo
# considerando únicamente productos de la misma categoría.
def predict_rating(user, producto_objetivo):
    # Obtener la categoría del producto objetivo
    categoria_objetivo = categorias[producto_objetivo]
    
    # Obtener las calificaciones del usuario
    user_ratings = df.loc[user]
    # Seleccionar productos que ya han sido calificados (valor diferente de 0)
    rated_items = user_ratings[user_ratings != 0]
    
    numerator = 0
    denominator = 0
    # Solo se consideran aquellos productos que pertenecen a la misma categoría
    for item, rating in rated_items.items():
        if categorias[item] == categoria_objetivo:
            similarity = item_similarity_df.loc[producto_objetivo, item]
            numerator += similarity * rating
            denominator += abs(similarity)
    
    if denominator == 0:
        return 0  # No hay productos calificados de la misma categoría
    return numerator / denominator

# Ejemplo de recomendación para el Usuario 'B'
user = 'B'
# Identificar los productos que el usuario B aún no ha calificado (valor 0)
productos_no_calificados = df.columns[df.loc[user] == 0]

predicciones = {}
for producto in productos_no_calificados:
    pred = predict_rating(user, producto)
    predicciones[producto] = pred

# Ordenar las predicciones de mayor a menor
recomendaciones = sorted(predicciones.items(), key=lambda x: x[1], reverse=True)

print(f"\nRecomendaciones para el usuario {user} (solo productos de la misma categoría):")
for producto, calificacion in recomendaciones:
    print(f"{producto}: calificación predicha {calificacion:.2f} - Categoría: {categorias[producto]}")

Información de productos con categorías:
             Categoría
Producto              
Producto1  Electrónica
Producto2       Cocina
Producto3   Literatura
Producto4         Moda
Producto5     Juguetes

Matriz de similitud entre productos:
           Producto1  Producto2  Producto3  Producto4  Producto5
Producto1   1.000000   0.886142   0.350329   0.087475   0.252050
Producto2   0.886142   1.000000   0.377964   0.136320   0.000000
Producto3   0.350329   0.377964   1.000000   0.601113   0.115470
Producto4   0.087475   0.136320   0.601113   1.000000   0.624695
Producto5   0.252050   0.000000   0.115470   0.624695   1.000000

Recomendaciones para el usuario B (solo productos de la misma categoría):
Producto2: calificación predicha 0.00 - Categoría: Cocina
Producto4: calificación predicha 0.00 - Categoría: Moda
